In [569]:
from bs4 import BeautifulSoup
import requests
import json
import io
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [570]:
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session = requests.Session()
session.mount('http://', adapter)
session.mount('https://', adapter)
main_page_url = 'https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie'
most_main_url = 'https://health-diet.ru'
main_page_response = requests.get(main_page_url)
main_page_src = main_page_response.text

In [20]:
#with io.open("index.html", "w", encoding="utf-8") as file:
 #   file.write(main_page_src)

In [571]:
with io.open("index.html", encoding="utf-8") as file:
    main_page_src = file.read()

In [572]:
main_page_soup = BeautifulSoup(main_page_src)

**Список ссылок и словарь (категория - ссылка) на страницы с товарами:**

In [573]:
categories = main_page_soup.find_all("div", class_ = "uk-grid uk-grid-medium")
products_url_lst = []
products_lst = []
for i in range(len(categories)-1):
    for j in categories[i].find_all("a"):
        products_lst.append(j.text)
        products_url_lst.append(j.get("href"))
prod_url_dict = dict(zip(products_lst, products_url_lst))
for i in products_lst:
    prod_url_dict[i] = most_main_url + prod_url_dict[i]
prod_url_dict["Полуфабрикаты"] = 'https://health-diet.ru/base_of_food/food_21253/'

**Импорируем словарь в json**

In [135]:
#with io.open("cats_links.json", "w", encoding="utf-8") as file:
#    json.dump(prod_url_dict, file, indent = 4, ensure_ascii = False)

**Получаем полный словарь данных (далее на всякий случаем сохраняем в json):**

In [574]:
prod_url_dict_new = prod_url_dict
for i in prod_url_dict_new:
    cat_page_soup_1 = BeautifulSoup(session.get(prod_url_dict_new[i]).text)
    lst_data = []
    sku_lst = []
    cal_lst = []
    protein_lst = []
    fats_lst = []
    carb_lst = []
    for j in cat_page_soup_1.find("body").find_all("td"):
        lst_data.append(j.text.strip())
    for j in range(len(lst_data)-1):
        if j % 5 == 0:
            sku_lst.append(lst_data[j])
    for j in range(1, len(lst_data)-1, 5):
        cal_lst.append(lst_data[j])
    for j in range(2, len(lst_data)-1, 5):
        protein_lst.append(lst_data[j])
    for j in range(3, len(lst_data)-1, 5):
        fats_lst.append(lst_data[j])
    for j in range(4, len(lst_data)-1, 5):
        carb_lst.append(lst_data[j])
    one_cat_dict_new = dict(zip(sku_lst, list(zip(cal_lst, protein_lst, fats_lst, carb_lst))))
    prod_url_dict_new[i] = one_cat_dict_new
for i in prod_url_dict_new:
    prod_url_dict_new[i].pop("СБАЛАНСИРОВАННАЯ МЕТОДИКА СНИЖЕНИЯ ВЕСА")
    prod_url_dict_new[i].pop("Дневник питания\nКонтролируйте своё питание и\nприобретайте полезные привычки")

In [411]:
#with io.open("all_data.json", "w", encoding="utf-8") as file:
    #json.dump(prod_url_dict_new, file, indent = 4, ensure_ascii = False)

**Получаем почти полную таблицу данных (не хватает категории товара):**

In [575]:
data = pd.DataFrame()
data["calory"] = []
data["protein"] = []
data["fats"] = []
data["carbohydrates"] = []
temp_lst = []
for i in prod_url_dict_new:
    for j in prod_url_dict_new[i]:
        temp_lst.append(list(prod_url_dict_new[i][j]))
for i in temp_lst:
    data.loc[len(data)] = i
prod_lst = [] 
for i in prod_url_dict_new:
    for j in prod_url_dict_new[i]:
        prod_lst.append(j)
data["products"] = prod_lst
data = data[["products", "calory", "protein", "fats", "carbohydrates"]]
data = data.drop_duplicates(subset = 'products')

**Получаем финальную таблицу:**

In [576]:
lst_i = []
lst_j = []
for i in prod_url_dict_new:
    for j in prod_url_dict_new[i]:
        lst_i.append(i)
        lst_j.append(j)
df2 = pd.DataFrame()
df2["category"] = lst_i
df2["products"] = lst_j
data = data.merge(df2, how = 'left')
data = data[["category", "products", "calory", "protein", "fats", "carbohydrates"]]
data

,category,products,calory,protein,fats,carbohydrates
0,Баранина и дичь,Антилопа,114 кКал,"22,38 г","2,03 г","2,03 г"
1,Баранина и дичь,"Антилопа, запеченная",150 кКал,"29,45 г","2,67 г","2,67 г"
2,Баранина и дичь,Баранина 1 кат.,209 кКал,"15,6 г","16,3 г","16,3 г"
3,Баранина и дичь,Баранина 2 кат.,166 кКал,"19,8 г","9,6 г","9,6 г"
4,Баранина и дичь,"Баранина австралийская ,голяшка, мясо и жир, о...",231 кКал,"25,25 г","13,69 г","13,69 г"
...,...,...,...,...,...,...
8389,Соусы и заправки,Соус универсальный,"211,1 кКал","8,8 г","17,8 г","17,8 г"
8390,Соусы и заправки,Соус хрен (со сметаной),"217,7 кКал","3,1 г","19,3 г","19,3 г"
8391,Соусы и заправки,Творожный майонез,"290,3 кКал","7,6 г","26,7 г","26,7 г"
8392,Соусы и заправки,Хлебный соус,"81,8 кКал","1,4 г","7,1 г","7,1 г"
